<a href="https://colab.research.google.com/github/vaswin0/tbClassification/blob/main/tbClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

from keras import layers
from keras.models import Model
import keras

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras import layers, models


In [ ]:
bb = np.load('/content/drive/MyDrive/tbData/b_phipt.npy').reshape((-1, 64, 64, 1))
tt = np.load('/content/drive/MyDrive/tbData/t_phipt.npy').reshape((-1, 64, 64, 1))

In [ ]:
X = np.append(bb, tt, axis=0)
Y = np.append(np.ones(len(bb)), np.zeros(len(tt))).reshape((-1, 1))


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [ ]:
# STEP 4: Build and Compile the CNN

input_img = layers.Input(shape=(64, 64, 1))

x = layers.Conv2D(8, (3,3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2,2), padding='same')(x)

x = layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((3,3), strides=(2,2), padding='valid')(x)

x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((3,3), strides=(1,1), padding='valid')(x)

x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2,2), strides=(2,2), padding='valid')(x)

x = layers.Conv2D(128, (3,3), strides=(2,2), activation='relu', padding='valid')(x)

x = layers.Flatten()(x)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(2, activation='softmax')(x)

model = models.Model(inputs=input_img, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 2, 2, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,198,978 (8.39 MB)

 Trainable params: 2,198,978 (8.39 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# STEP 5: Train the Model
history = model.fit(
    X_train, Y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_test, Y_test),
    verbose=1
)


Epoch 1/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8013 - loss: 0.3806 - val_accuracy: 0.8914 - val_loss: 0.2595
Epoch 2/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8968 - loss: 0.2469 - val_accuracy: 0.8970 - val_loss: 0.2477
Epoch 3/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9024 - loss: 0.2361 - val_accuracy: 0.8968 - val_loss: 0.2621
Epoch 4/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9057 - loss: 0.2303 - val_accuracy: 0.9054 - val_loss: 0.2323
Epoch 5/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9074 - loss: 0.2273 - val_accuracy: 0.9049 - val_loss: 0.2362
Epoch 6/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9104 - loss: 0.2194 - val_accuracy: 0.9080 - val_loss: 0.2287
Epoch 7/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9108 - loss: 0.2139 - val_accuracy: 0.9018 - val_loss: 0.2481
Epoch 8/30
1117/1117 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9122 - loss: 0.2104 -

In [ ]:
# STEP 6: Evaluate and Save the Model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_acc:.4f}")

model.save('/content/tip_body_model.h5')

# Optional: Download the model to your local machine
from google.colab import files
files.download('/content/tip_body_model.h5')


280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8980 - loss: 0.2441


Test Accuracy: 0.9013


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
len(X)

44674

In [ ]:
len(X_train)

35739